In [4]:
import os
from datetime import datetime
from typing import List, Optional

import googlemaps
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

True

In [5]:
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon


def get_atlanta_polygon():
    # Approximate polygon for Atlanta city limits
    # These coordinates form a rough boundary of Atlanta
    atlanta_coords = [
        (-84.551068, 33.657330),
        (-84.289656, 33.657330),
        (-84.289656, 33.886823),
        (-84.551068, 33.886823),
    ]
    return Polygon(atlanta_coords)


def is_in_atlanta(lat, lng):
    point = Point(lng, lat)
    atlanta_polygon = get_atlanta_polygon()
    return atlanta_polygon.contains(point)


class Review(BaseModel):
    author: str
    rating: float
    text: str
    time: str


class PlaceDetails(BaseModel):
    name: str
    address: str
    rating: Optional[float] = None
    price_level: Optional[int] = None
    reviews: List[Review] = []
    opening_hours: Optional[List[str]] = None
    is_restaurant: bool = Field(
        default=False, description="Indicates if the place is a restaurant"
    )
    is_in_atlanta: bool = Field(
        default=False, description="Indicates if the place is in Atlanta"
    )
    distance_from_base: float = Field(
        default=0.0, description="Distance from base location in kilometers"
    )
    types: List[str] = Field(default_factory=list, description="Types of the place")
    website: Optional[str] = None
    phone_number: Optional[str] = None


def create_gmaps_client(api_key):
    return googlemaps.Client(key=api_key)


def get_restaurants_in_atlanta(
    gmaps,
    base_location: str,
    name: str = None,
    cuisine_type: str = None,
    max_distance: float = 10,  # in kilometers
    limit: int = 20,
    min_rating: float = 0,  # New parameter for minimum rating
) -> List[PlaceDetails]:
    geolocator = Nominatim(user_agent="myapp")
    base_coords = geolocator.geocode(base_location + ", Atlanta, GA")
    if not base_coords:
        raise ValueError("Base location not found")

    base_point = (base_coords.latitude, base_coords.longitude)

    query = "restaurant"
    if name:
        query += f" {name}"
    if cuisine_type:
        query += f" {cuisine_type}"

    places_result = gmaps.places(
        query=query, location=base_point, radius=max_distance * 1000
    )

    results = []
    for place in places_result.get("results", []):
        lat = place["geometry"]["location"]["lat"]
        lng = place["geometry"]["location"]["lng"]

        if not is_in_atlanta(lat, lng):
            continue

        place_point = (lat, lng)
        distance = geodesic(base_point, place_point).kilometers

        if distance > max_distance:
            continue

        place_id = place["place_id"]

        place_details = gmaps.place(
            place_id=place_id,
            fields=[
                "name",
                "formatted_address",
                "type",
                "rating",
                "reviews",
                "price_level",
                "opening_hours",
                "website",
                "formatted_phone_number",
            ],
        )

        result = place_details["result"]

        # Skip this place if its rating is below the minimum
        if result.get("rating", 0) < min_rating:
            continue

        details = PlaceDetails(
            name=result.get("name", ""),
            address=result.get("formatted_address", ""),
            rating=result.get("rating"),
            price_level=result.get("price_level"),
            opening_hours=result.get("opening_hours", {}).get("weekday_text"),
            is_restaurant=True,
            is_in_atlanta=True,
            distance_from_base=round(distance, 2),
            types=place.get("types", []),
            website=result.get("website"),
            phone_number=result.get("formatted_phone_number"),
        )

        if "reviews" in result:
            details.reviews = [
                Review(
                    author=review.get("author_name", ""),
                    rating=review.get("rating", 0),
                    text=review.get("text", ""),
                    time=datetime.fromtimestamp(review.get("time", 0)).strftime(
                        "%Y-%m-%d %H:%M:%S"
                    ),
                )
                for review in result["reviews"]
            ]

        results.append(details)

        if len(results) >= limit:
            break

    return results

In [7]:
# Example usage:
api_key = os.getenv("GOOGLE_MAPS_API_KEY")
gmaps_client = create_gmaps_client(api_key)
results = get_restaurants_in_atlanta(
    gmaps_client,
    base_location="Georgia Tech",
    name="blaze",
    cuisine_type="Italian",
    max_distance=5,
    limit=10
)

for result in results:
    print(f"{result.name} - {result.address} - {result.distance_from_base}mi")

Blaze Pizza - 540 17th St NW, Atlanta, GA 30318, USA - 1.82mi
